# Завдання класифікації тексту

У цьому модулі ми розпочнемо з простого завдання класифікації тексту на основі набору даних **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: ми будемо класифікувати заголовки новин у одну з 4 категорій: Світ, Спорт, Бізнес та Наука/Технології.

## Набір даних

Для завантаження набору даних ми скористаємося API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Тепер ми можемо отримати доступ до навчальної та тестової частин набору даних, використовуючи `dataset['train']` та `dataset['test']` відповідно:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Давайте виведемо перші 10 нових заголовків з нашого набору даних:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Векторизація тексту

Тепер нам потрібно перетворити текст у **числа**, які можна представити як тензори. Якщо ми хочемо представлення на рівні слів, необхідно виконати два кроки:

* Використати **токенізатор**, щоб розбити текст на **токени**.
* Побудувати **словник** цих токенів.

### Обмеження розміру словника

У прикладі з набором даних AG News розмір словника досить великий — понад 100 тисяч слів. Загалом, нам не потрібні слова, які рідко зустрічаються в тексті — лише кілька речень їх міститимуть, і модель не зможе навчитися на них. Тому має сенс обмежити розмір словника до меншого числа, передавши аргумент до конструктора векторизатора:

Обидва ці кроки можна виконати за допомогою шару **TextVectorization**. Давайте створимо об'єкт векторизатора, а потім викличемо метод `adapt`, щоб пройти через весь текст і побудувати словник:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Примітка** ми використовуємо лише підмножину всього набору даних для створення словника. Ми робимо це, щоб прискорити час виконання і не змушувати вас чекати. Однак, ми ризикуємо, що деякі слова з усього набору даних не будуть включені до словника і будуть проігноровані під час навчання. Таким чином, використання повного розміру словника і проходження через весь набір даних під час `adapt` може підвищити кінцеву точність, але не значно.

Тепер ми можемо отримати доступ до фактичного словника:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Використовуючи векторизатор, ми можемо легко закодувати будь-який текст у набір чисел:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Представлення тексту за методом "Мішок слів"

Оскільки слова передають значення, іноді ми можемо зрозуміти сенс тексту, просто дивлячись на окремі слова, незалежно від їхнього порядку в реченні. Наприклад, при класифікації новин слова, такі як *погода* і *сніг*, ймовірно, вказуватимуть на *прогноз погоди*, тоді як слова *акції* і *долар* будуть відноситися до *фінансових новин*.

**Мішок слів** (BoW) — це найпростіше для розуміння традиційне представлення векторів. Кожне слово пов'язане з індексом вектора, а елемент вектора містить кількість появ кожного слова в даному документі.

![Зображення, яке показує, як представлення вектора "Мішок слів" зберігається в пам'яті.](../../../../../translated_images/uk/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Ви також можете думати про BoW як про суму всіх векторів з одним активним елементом (one-hot-encoded) для окремих слів у тексті.

Нижче наведено приклад того, як створити представлення "Мішок слів" за допомогою бібліотеки Scikit Learn для Python:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Ми також можемо використовувати векторизатор Keras, який ми визначили вище, перетворюючи кожен номер слова в one-hot кодування і додаючи всі ці вектори разом:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Примітка**: Ви можете здивуватися, що результат відрізняється від попереднього прикладу. Причина полягає в тому, що в прикладі з Keras довжина вектора відповідає розміру словника, який був створений на основі всього набору даних AG News, тоді як у прикладі з Scikit Learn ми створили словник на основі тексту зразка безпосередньо під час виконання.


## Навчання класифікатора BoW

Тепер, коли ми навчилися створювати представлення тексту у вигляді "мішка слів", давайте навчимо класифікатор, який його використовує. Спочатку нам потрібно перетворити наш набір даних у представлення "мішка слів". Це можна зробити за допомогою функції `map` наступним чином:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Тепер давайте визначимо просту нейронну мережу-класифікатор, яка містить один лінійний шар. Розмір вхідних даних — `vocab_size`, а розмір вихідних даних відповідає кількості класів (4). Оскільки ми вирішуємо задачу класифікації, остаточна функція активації — **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Оскільки у нас є 4 класи, точність понад 80% є хорошим результатом.

## Навчання класифікатора як однієї мережі

Оскільки векторизатор також є шаром Keras, ми можемо визначити мережу, яка включає його, і навчати її від початку до кінця. Таким чином, нам не потрібно векторизувати набір даних за допомогою `map`, ми можемо просто передати оригінальний набір даних на вхід мережі.

> **Примітка**: Нам все одно доведеться застосовувати `map` до нашого набору даних, щоб перетворити поля зі словників (таких як `title`, `description` і `label`) у кортежі. Однак, завантажуючи дані з диска, ми можемо одразу створити набір даних із необхідною структурою.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Біграми, триграми та n-грами

Одним із обмежень підходу "мішок слів" є те, що деякі слова є частиною багатослівних виразів. Наприклад, слово "гарячий хот-дог" має зовсім інше значення, ніж слова "гарячий" і "пес" у інших контекстах. Якщо ми завжди представлятимемо слова "гарячий" і "пес" за допомогою однакових векторів, це може заплутати нашу модель.

Щоб вирішити цю проблему, у методах класифікації документів часто використовуються **n-грамні представлення**, де частота кожного слова, пари слів або трійки слів є корисною ознакою для навчання класифікаторів. Наприклад, у біграмному представленні ми додамо до словника всі пари слів, окрім початкових слів.

Нижче наведено приклад того, як створити біграмне представлення "мішка слів" за допомогою Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Основний недолік підходу n-грам полягає в тому, що розмір словника починає зростати надзвичайно швидко. На практиці нам потрібно поєднувати представлення n-грам із технікою зменшення розмірності, такою як *вбудовування* (embeddings), про яку ми поговоримо в наступному розділі.

Щоб використовувати представлення n-грам у нашому наборі даних **AG News**, нам потрібно передати параметр `ngrams` до конструктора `TextVectorization`. Довжина словника біграм є **значно більшою**, у нашому випадку це понад 1,3 мільйона токенів! Тому має сенс обмежити кількість токенів біграм до якогось розумного числа.

Ми могли б використати той самий код, що й вище, для навчання класифікатора, однак це було б дуже неефективно з точки зору пам’яті. У наступному розділі ми будемо навчати класифікатор біграм, використовуючи вбудовування. Тим часом ви можете експериментувати з навчанням класифікатора біграм у цьому ноутбуці та перевірити, чи зможете досягти вищої точності.


## Автоматичний розрахунок векторів BoW

У наведеному вище прикладі ми обчислювали вектори BoW вручну, підсумовуючи one-hot кодування окремих слів. Однак, остання версія TensorFlow дозволяє автоматично обчислювати вектори BoW, передаючи параметр `output_mode='count` до конструктора векторизатора. Це значно спрощує визначення та навчання нашої моделі:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Частота терміну - обернена частота документа (TF-IDF)

У представленні BoW (мішок слів) частота появи слів зважується однаково незалежно від самого слова. Однак очевидно, що часто вживані слова, такі як *a* і *in*, набагато менш важливі для класифікації, ніж спеціалізовані терміни. У більшості завдань обробки природної мови деякі слова є більш релевантними, ніж інші.

**TF-IDF** означає **частота терміну - обернена частота документа**. Це варіація мішка слів, де замість двійкового значення 0/1, яке вказує на появу слова в документі, використовується значення з плаваючою точкою, що пов'язане з частотою появи слова в корпусі.

Більш формально, вага $w_{ij}$ слова $i$ в документі $j$ визначається як:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
де
* $tf_{ij}$ — кількість появ $i$ в $j$, тобто значення BoW, яке ми бачили раніше
* $N$ — кількість документів у колекції
* $df_i$ — кількість документів, що містять слово $i$ у всій колекції

Значення TF-IDF $w_{ij}$ збільшується пропорційно до кількості разів, коли слово з'являється в документі, і зменшується залежно від кількості документів у корпусі, які містять це слово. Це допомагає врахувати той факт, що деякі слова з'являються частіше за інші. Наприклад, якщо слово з'являється в *кожному* документі колекції, $df_i=N$, і $w_{ij}=0$, такі терміни будуть повністю ігноровані.

Ви можете легко створити векторизацію TF-IDF тексту за допомогою Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

У Keras шар `TextVectorization` може автоматично обчислювати частоти TF-IDF, передаючи параметр `output_mode='tf-idf'`. Давайте повторимо код, який ми використовували вище, щоб перевірити, чи підвищує використання TF-IDF точність:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Висновок

Хоча представлення TF-IDF надають ваги частотам різних слів, вони не здатні передати значення чи порядок. Як сказав відомий лінгвіст Дж. Р. Ферт у 1935 році: "Повне значення слова завжди є контекстуальним, і жодне вивчення значення поза контекстом не може бути сприйняте серйозно." Пізніше в курсі ми дізнаємося, як захоплювати контекстуальну інформацію з тексту за допомогою мовного моделювання.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
